# NST for videos

In [1]:
import numpy as np
import os
import cv2
import tensorflow as tf
from google.colab import files

!git clone https://github.com/TeamCV3/video-style-transfer -q
%cd video-style-transfer/custom_nst
uploaded_file = None
from custom_stylization_model import CustomStylizationModel

/content/video-style-transfer/custom_nst


In [27]:
"""
This is the main function that applies our Neural Network framewise,
and concatenates resulting frames in a film.
Further, it applies optical flow between frames for less noisy picture.
"""

def style_video(video, style):

  # function, that gets n-th digit (used for naming purposes)
  def get_digit(number, n):
    return number // 10**n % 10
  
  def get_mask_changes(prev, current):
    prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    current = cv2.cvtColor(current, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(prev.shape)

    mask[..., 1] = 255

    # Calculates dense optical flow by Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev, current,
            None,
            0.5, 3, 13, 3, 5, 1.2, 0)
    
    # Computes the magnitude and angle of the 2D vectors
    magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    
    # magnitude (normalized)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX) / 255
    rgb = np.dstack([magnitude,magnitude,magnitude])

    return rgb

  """
  Split target video into frames and load the frames to a folder
  """
  vidcap = cv2.VideoCapture(video)
  success,image = vidcap.read()
  count = 0
  print("Frames upload started...")
  frames = []
  while success:
    # used for naming
    tt = get_digit(count, 4)
    t = get_digit(count, 3)
    h = get_digit(count, 2)
    ts = get_digit(count, 1)
    o = get_digit(count, 0)
    number = str(tt) + str(t) + str(h) + str(ts) + str(o)
    # save a frame
    frames.append(image)     
    success,image = vidcap.read()
    count += 1
  print(count,"frames uploaded")

  """
  Apply style transfer frame-wise. Here, to overcome colab limitations, we use less FPS
  and less iterations of our model.
  """
  i = 0 
  print("Style started...")
  results = []

  for filename in frames:
    if i%2==0:
      # call the model
      if i > 1:
        original = frames[i]
        style_img = results[i//2-1]
        previous = frames[i-2]
        mask = get_mask_changes(previous,original)
        new = (style_img * (1 - mask) + original * mask).astype(np.uint8)
        cv2.imwrite(f"frame_style.jpg", new) 
      else:
        cv2.imwrite(f"frame_style.jpg", filename) 
      model = CustomStylizationModel(
          f'frame_style.jpg',
          style,
          content_layers=['block5_conv2'],
          style_layers=['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1'],
          style_weight=1e-3,
          content_weight=1e3*5,
          total_variation_weight=30,
          n_iter=100,
          opt=tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
      )
      result = model.compose()
      results.append(np.array(result)[:, :, ::-1].copy())
    i+=1
    # print progress
    if i%100==0:
      print(np.round(i/len(frames),2)*100,"% complete...")
  
  """
  Saving frames into videos
  """
  frames_r = frames[0::2]

  height, width, layers = frames_r[0].shape
  fourcc = cv2.VideoWriter_fourcc(*'XVID')

  # standard video of styled images
  video_std_name = 'result_std.mp4'
  video_std = cv2.VideoWriter(video_std_name,fourcc, 12, (width,height))
  # video of styled images + optical flow
  video_opt_flow_name = 'result_opt_flow.mp4'
  video_opt_flow = cv2.VideoWriter(video_opt_flow_name,fourcc, 12, (width,height))
  
  # write frames to standard video
  for img in frames_r:
    video_std.write(img)
  video_std.release()

  for img in results:
    video_opt_flow.write(img)
  video_opt_flow.release() 

In [4]:
video = files.upload()
video_filename = next(iter(video))

Saving scene_1.mp4 to scene_1.mp4


In [5]:
style = files.upload()
style_filename = next(iter(style))

Saving style (1).png to style (1).png


In [28]:
style_video(video_filename, style_filename)

Frames upload started...
92 frames uploaded
Style started...
